In [1]:
import pygame
import os 
import random

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.init()                                          

pygame.mixer.init()                                     

pygame.font.init()                                      

# Gives the game window a title
pygame.display.set_caption("Highway Dash")

In [3]:
width, height = 1530, 750

# Sets window to specified size
screen = pygame.display.set_mode((width,height)) 

# RGB color code
black = (0, 0, 0)   
red = (248,36,12)
yellow = (255, 255, 0)
white = (255, 255, 255)
grey = (128, 128, 128)
green = (0, 255, 0)
blue = (0, 0, 255)

In [4]:
# Import car images
red_car = pygame.image.load(os.path.join('car assets','car', 'red main car.png'))
black_car = pygame.image.load(os.path.join('car assets','car', 'black car.png'))
taxi_car = pygame.image.load(os.path.join('car assets','car', 'taxi car.png'))
purple_car = pygame.image.load(os.path.join('car assets','car', 'purple car.png'))
green_car = pygame.image.load(os.path.join('car assets','car', 'green car.png'))
tan_car = pygame.image.load(os.path.join('car assets','car', 'short tan car.png'))

explosion = pygame.image.load(os.path.join('car assets','car', 'explosion.png'))
smoke = pygame.transform.scale(pygame.image.load(os.path.join('car assets','car', 'smoke.png')), (100,150))
fire = pygame.transform.scale(pygame.image.load(os.path.join('car assets','car', 'fire.png')), (100,90))

car_crash = pygame.mixer.Sound(os.path.join('car assets', 'sound', 'crash.mp3'))
car_explode = pygame.mixer.Sound(os.path.join('car assets', 'sound', 'Grenade+1.mp3'))

# Initialize rectangle representing the player's car
taxi = pygame.Rect(width // 2 - (taxi_car.get_width() // 2 - 5), height - 250, taxi_car.get_width(), taxi_car.get_height())

In [5]:
# Variables to draw the background 
road = pygame.Rect(width // 2 - 500, 0 ,1000, height)
left_border = pygame.Rect(width // 2 - 500, 0, 10, height)
right_border = pygame.Rect(width // 2 + 500, 0, 10, height)

In [6]:
car_positions = []

def spawn_cars():
   
    car_images = [black_car, red_car, purple_car, green_car, tan_car]

   
    car_image = random.choice(car_images)

    car_width = car_image.get_width() 
    car_height = car_image.get_height() 

    # Center of lanes
    lane_positions = [370, 570, 770, 970, 1170]  

    
    lane_x = random.choice(lane_positions)

    car_x = lane_x - (car_width // 2)
       
    car_y = random.randint(-500, -100)

    # Create a rect object for the car position
    car_rect = pygame.Rect(car_x, car_y, car_width, car_height)

    # Check for collision with other cars
    for car, rect in car_positions: 
        if car_rect.colliderect(rect):
            # Adjust position if there's a collision
            car_rect.y -= (3 * rect.height)
            
    car_positions.append((car_image, car_rect))

In [7]:
def update_car_positions(road_speed):
  for car in car_positions:
    car_rect = car[1]  # Get the Rect object representing the car's position
    car_rect.y += road_speed  # Increment y-coordinate to move the car downwards

In [8]:
def draw_lane(add_x_coor, road_y):
    lane_y_coor = road_y % 100
    for i in range(height//100):
        pygame.draw.rect(screen, white, (pygame.Rect((width // 2 - 500) + add_x_coor, lane_y_coor + i * 100, 10, 50)))

In [9]:
def game_screen(taxi, road_y, score_text, lives_text, speed_text):
    screen.fill(green)
    pygame.draw.rect(screen, grey, road)
    pygame.draw.rect(screen, white, left_border)
    pygame.draw.rect(screen, white, right_border)
    draw_lane(200, road_y)
    draw_lane(400, road_y)
    draw_lane(600, road_y)
    draw_lane(800, road_y)
    screen.blit(taxi_car, (taxi.x, taxi.y))
    screen.blit(score_text, (10, 10))
    screen.blit(lives_text, (10, 80))
    screen.blit(speed_text, (10, 160))
    # For each element in the list, the images are pasted with the corresponding position
    for car in car_positions:
        screen.blit(car[0], car[1])

In [10]:
def taxi_movement(taxi):       
    keys = pygame.key.get_pressed()
    if (keys[pygame.K_LEFT] or keys[pygame.K_a]) and (taxi.x > (left_border.x + 200)):            
        taxi.x -= 200
    elif (keys[pygame.K_RIGHT] or keys[pygame.K_d]) and (taxi.x < (right_border.x - 200)):    
        taxi.x += 200          

In [11]:
def display_leaderboard(leaderboard):
    screen.fill(black)
    leaderboard_font = pygame.font.SysFont('Arial',100)
    title_text = leaderboard_font.render('Leaderboard', 1, white)
    screen.blit(title_text, ((width - title_text.get_width()) // 2, 25))

    name_font = pygame.font.SysFont('Arial', 60)
    y_position = 200
    for rank, (name, score) in enumerate(sorted(leaderboard.items(), key=lambda x: x[1], reverse=True), 1):
        entry_text = name_font.render(f"{rank}. {name}: {score:.0f}", 1, white)
        screen.blit(entry_text, ((width - entry_text.get_width()) // 2, y_position))
        y_position += 70

    # Update display
    pygame.display.flip()

In [12]:
def get_player_name():
    name = ""
    font = pygame.font.SysFont('Arial', 40)
    input_rect = pygame.Rect((width // 2) - 200, (height // 2) - 25, 400, 50)
    active = False
    color = white
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if input_rect.collidepoint(event.pos):
                    active = True
                else:
                    active = False
                color = blue if active else white
            if event.type == pygame.KEYDOWN:
                if active:
                    if event.key == pygame.K_RETURN:
                        return name
                    elif event.key == pygame.K_BACKSPACE:
                        name = name[:-1]
                    else:
                        name += event.unicode 
        screen.fill(black)
        pygame.draw.rect(screen, color, input_rect, 2)
        text_surface = font.render(name, 1, white)
        screen.blit(text_surface, (input_rect.x + 5, input_rect.y + 5))
        
        enter_name_text = font.render("Enter name", 1, white)
        screen.blit(enter_name_text, ((width // 2 - enter_name_text.get_width() // 2), (height // 2 - 80)))
        
        pygame.display.flip()

In [13]:
def load_leaderboard():
    board = {}
    if os.path.exists("leaderboard.txt"):
        with open("leaderboard.txt", "r") as file:
            for line in file:
                name, score = line.strip().split(",")
                board[name] = float(score)
    return board

In [14]:
def save_leaderboard(leaderboard):
    with open("leaderboard.txt", "w") as file:
        for name, score in leaderboard.items():
            file.write(f"{name},{score}\n")

In [15]:
def main_funcs():

    music = pygame.mixer.music.load(os.path.join('car assets', 'sound', 'music1.mp3'))

    pygame.mixer.music.set_volume(0.1)

    pygame.mixer.music.play(loops=-1)

    clock = pygame.time.Clock()

    run = True

    # Lanes do not move yet 
    road_y = 0

    road_speed = 0  

    animation_started = False

    timer = pygame.time.get_ticks()
    
    score = 0

    score_font = pygame.font.SysFont('Arial', 50)

    speed_font = pygame.font.SysFont('Arial', 35)

    fscore_font = pygame.font.SysFont('Arial', 150)

    times_hit = 3

    spawn_rate = 0.0112

    collision_detected = False

    leaderboard = load_leaderboard() 

    player_name = get_player_name() 
    
    while run:

        # Clock.tick() controls frame rate and returns time since last frame in milliseconds. It gets the time passed since last frame
        fps = clock.tick(120)

        # Checks for different events that occur in the game
        for event in pygame.event.get():

            # Ends the while loop; If you hit the 'x' at the top right corner you exit the game
            if event.type == pygame.QUIT:
                run = False
            # If key is pressed, car will move and road animation will start
            if event.type == pygame.KEYDOWN:
                if not animation_started:
                    animation_started = True
                    road_speed = 3
                taxi_movement(taxi)
                
        if animation_started:
            road_speed += 0.002  
            score += 1/15
 
            # There's a 1.1% chance the random number will be less than 0.011. Anytime that happens a car will spawn
            if random.random() < spawn_rate: 
                spawn_cars()
                if road_speed < 20:
                    spawn_rate = spawn_rate
                elif road_speed >= 20:
                    spawn_rate = 0.013
                elif road_speed >= 50:
                    spawn_rate = 0.014
                elif road_speed >= 100:
                    spawn_rate = 0.015
                collision_detected = False
        # Check for collision with player's car
        for car, rect in car_positions:
            if taxi.colliderect(rect):
                if collision_detected == False:
                    collision_detected = True
                    times_hit -= 1 
                    taxi.y += 100
                    timer = pygame.time.get_ticks()
                   
                    if times_hit != 0:
                        car_crash.play()
                    
                    if score <= 100:
                        road_speed = 1
                    elif 100 <= score <= 500:
                        road_speed = 3
                    elif score > 500:
                        road_speed = 10
                    break 
                        
        if pygame.time.get_ticks() - timer >= 3000:
            taxi.y = height - 250
            
        road_y += road_speed  
        update_car_positions(road_speed)  

        score_text = score_font.render(f'Score: {score:.0f}', 1, white)
        lives_text = score_font.render('Lives: ' + str(times_hit), 1, white)
        speed_text = speed_font.render(f'Speed: {3*(road_speed):.2f} mph', 1, white)
            
        game_screen(taxi, road_y, score_text, lives_text, speed_text)


        if times_hit == 2:
            screen.blit(smoke, (taxi.x+30, taxi.y - taxi_car.get_height()+40))

        elif times_hit == 1:
            screen.blit(fire, (taxi.x-9, (taxi.y - taxi_car.get_height()+70)))

        elif times_hit == 0:
            car_explode.play()
            screen.blit(explosion, (taxi.x - (explosion.get_width()//3), taxi.y - (explosion.get_height()//3)))
            final_score = fscore_font.render(f'Final Score: {score:.0f}', 1, red)
            screen.blit(final_score, (width // 2 - final_score.get_width() // 2, height // 2 - final_score.get_height() // 2))
            pygame.display.update()
            pygame.time.delay(3000)
            
            if player_name not in leaderboard or score > leaderboard[player_name]:
                leaderboard[player_name] = score  
                save_leaderboard(leaderboard)
            display_leaderboard(leaderboard)

            pygame.time.delay(3000)  

            score = 0
            times_hit = 3
            animation_started = False
            road_y = 0
            road_speed = 0
            car_positions.clear()
            taxi.y = height - 250
            taxi.x = (width // 2) - (taxi_car.get_width() // 2 - 5)

        if player_name in leaderboard and score > leaderboard[player_name]:
            new_high_score_text = speed_font.render('New High Score!', 1, white)
            screen.blit(new_high_score_text, (10, 210))
            pygame.display.update()

        pygame.display.update()

In [16]:
main_funcs()

In [17]:
pygame.quit()